# Imports

In [6]:
import polars as pl
import unicodedata
import re

# Read data

In [ ]:
def read_metro_conexion():

    df = pl.read_csv("/home/onyxia/work/hackathon_mobilites_2025/data/metro_connexion_corresp_idfm_ref.csvin")
    return df

In [8]:
def normalize_column_name(name: str) -> str:
    # Mettre en minuscules
    name = name.lower()
    # Supprimer les accents
    name = unicodedata.normalize("NFKD", name)
    name = name.encode("ascii", "ignore").decode("utf-8")
    # Remplacer tout caractère non a-z par _
    name = re.sub(r'[^a-z]+', '_', name)
    # Retirer les underscores en trop (début/fin)
    name = name.strip('_')
    return name

In [9]:
df = read_metro_conexion()
df = df.rename({col: normalize_column_name(col) for col in df.columns})
df.head()

href,provenance,destination,station,description_detaillee,ligne_de_provenance,ligne_de_destination,id_zone_arret_icar,id_ligne_de_destination,id_ligne_de_provenance
str,str,str,str,str,str,str,i64,str,str
"""https://www.metro-connexion.or…","""Champ de Mars, Pontoise, Versa…","""Mairie de Montreuil""","""Alma - Marceau Pont de l'Alma""","""Via la station Alma-Marceau. …","""C""","""9""",42182,"""C01379""","""C01727"""
"""https://www.metro-connexion.or…","""Champ de Mars, Pontoise, Versa…","""Champ de Mars, Pontoise, Versa…","""Alma - Marceau Pont de l'Alma""","""La correspondance pour retourn…","""C""","""C""",42182,"""C01727""","""C01727"""
"""https://www.metro-connexion.or…","""Champ de Mars, Pontoise, Versa…","""Pont de Sèvres""","""Alma - Marceau Pont de l'Alma""","""Via la station Alma-Marceau. …","""C""","""9""",42182,"""C01379""","""C01727"""
"""https://www.metro-connexion.or…","""Choisy-le-Roi, Massy-Palaiseau…","""Mairie de Montreuil""","""Alma - Marceau Pont de l'Alma""","""Via la station Alma-Marceau. …","""C""","""9""",42182,"""C01379""","""C01727"""
"""https://www.metro-connexion.or…","""Choisy-le-Roi, Massy-Palaiseau…","""Choisy-le-Roi, Massy-Palaiseau…","""Alma - Marceau Pont de l'Alma""","""La correspondance pour retourn…","""C""","""C""",42182,"""C01727""","""C01727"""


In [10]:
#Un exemple : il semblerait que les étapes sont séparées par des retours à la ligne
print(repr(df['description_detaillee'][0]))
print(df['description_detaillee'][0])

"Via la station Alma-Marceau.\r\nLa correspondance du RER C Pont de l'Alma vers le métro Alma-Marceau va se faire par l'extérieur en traversant le pont de l'Alma.\r\n\r\nEn tête, passer les composteurs situés contre le mur.\r\nMonter le double escalier. On arrive à l'extérieur en léger contrebas d'un terre-plein qui se trouve face à nous. Pour monter sur le terre-plein, il y a quatre marches en sortant du RER en provenance de Champ de Mars et une seule marche en sortant du RER en provenance de Bibliothèque François Mitterrand. Ce terre-plein est lui-même légèrement en côte en allant vers la Seine. Il forme une sorte de triangle qui se présente comme suit :Devant, les marches permettant de monter sur le terre-plein, (la base du triangle)De 9 H, à 11 H, un autre terre-plein en terre en contrebas que l'on atteint par un court escalier descendant, (le côté gauche du triangle).\r\nEntre 9 H et 3 H, vers le fond, à une quarantaine de mètres, la SeineA 11 H, le pont de l'alma enjambe la Seine

### quelques exemples de provenance et destinations; il s'agit uniquement des terminus des différentes lignes

In [11]:
df.select(["provenance", "ligne_de_provenance"]).unique().sort("ligne_de_provenance")

provenance,ligne_de_provenance
str,str
"""La Défense Grande Arche""","""1"""
"""Château de Vincennes""","""1"""
"""Boulogne""","""10"""
"""Gare d'Austerlitz""","""10"""
"""Chatelet""","""11"""
…,…
"""Porte D'Asnières - Marguerite …","""T3b"""
"""Gare de Noisy-le-Sec""","""T4"""
"""Bondy""","""T4"""


In [12]:
df.select(["destination", "ligne_de_destination"]).unique().sort("ligne_de_destination")

destination,ligne_de_destination
str,str
"""Château de Vincennes""","""1"""
"""La Défense Grande Arche""","""1"""
"""Boulogne""","""10"""
"""Gare d'Austerlitz""","""10"""
"""Mairie des Lilas""","""11"""
…,…
"""Porte de Vincennes""","""T3b"""
"""Porte D'Asnières - Marguerite …","""T3b"""
"""Gare de Noisy-le-Sec""","""T4"""


# Create a column with the number of steps in each connecting commute 

In [13]:
df = df.with_columns(
    pl.col("description_detaillee")
      .str.count_matches(r"\r\n")
      .alias("nb_etapes")
)

# Group by connecting stations

In [14]:
result = (
    df.group_by("station")
      .agg([
          pl.col("provenance").n_unique().alias("nb_provenance"),
          pl.concat_str(["ligne_de_provenance", "provenance"], separator="-").unique().alias("distinct_provenance"),
          pl.col("destination").n_unique().alias("nb_destination"),
          pl.concat_str(["ligne_de_destination", "destination"], separator="-").unique().alias("distinct_destination"),
          pl.col("nb_etapes").sum().alias("total_nb_etapes")
      ])
)
result.sort('nb_provenance', descending=True)

station,nb_provenance,distinct_provenance,nb_destination,distinct_destination,total_nb_etapes
str,u32,list[str],u32,list[str],u32
"""Châtelet""",15,"[""B-Aulnay-sous-Bois, Aéroport Charles de Gaulle, Mitry-Claye"", ""4-Bagneux - Lucie Aubrac"", … ""A-Vincennes, Boissy-St-Léger, Marne-la-Vallée""]",15,"[""4-Porte de Clignancourt"", ""1-Château de Vincennes"", … ""B-Bourg-la-Reine, Robinson, St-Rémy-lès-Chevreuse""]",9636
"""Saint-Lazare""",11,"[""T1-Asnières-Genevilliers - Les Courtilles"", ""T6-Châtillon - Montrouge"", … ""9-Pont de Sèvres""]",11,"[""3-Galiéni"", ""12-Mairie d'Aubervilliers"", … ""9-Mairie de Montreuil""]",3990
"""République""",10,"[""8-Balard"", ""5-Bobigny"", … ""9-Pont de Sèvres""]",10,"[""3-Galiéni"", ""5-Bobigny"", … ""8-Créteil - Pointe du Lac""]",1645
"""Gare de Lyon""",8,"[""1-Château de Vincennes"", ""1-La Défense Grande Arche"", … ""A-Vincennes, Boissy-St-Léger, Marne-la-Vallée""]",8,"[""D-Villeneuve-Saint-Georges, Melun, Malesherbes"", ""A-Vincennes, Boissy-St-Léger, Marne-la-Vallée"", … ""1-La Défense Grande Arche""]",1364
"""Montparnasse - Bienvenüe""",8,"[""T1-Asnières-Genevilliers - Les Courtilles"", ""4-Bagneux - Lucie Aubrac"", … ""4-Porte de Clignancourt""]",8,"[""4-Porte de Clignancourt"", ""12-Mairie d'Aubervilliers"", … ""T6-Châtillon - Montrouge""]",1317
…,…,…,…,…,…
"""Haussemann""",1,"[""E-Chelles, Villiers-sur-Marne""]",12,"[""T1-Asnières-Genevilliers - Les Courtilles"", ""9-Pont de Sèvres"", … ""3-Pont de Levallois""]",2461
"""Boulogne - Pont de Saint-Cloud""",1,"[""10-Boulogne""]",2,"[""T2-Porte de Versailles, Parce des expositions"", ""T2-Pont de Bezons""]",59
"""Pré Saint-Gervais""",1,"[""7B-Louis Blanc""]",3,"[""T3b-Porte de Vincennes"", ""T3b-Porte D'Asnières - Marguerite Long"", ""7B-Louis Blanc""]",110


In [15]:
result.write_parquet("/home/onyxia/work/hackathon_mobilites_2025/data/interim/metro_connexion_nombre_etape.parquet")